In [1]:
# 1. __getattr__
# 作用: 如果属性查找（attribute lookup）在实例以及对应的类中（通过__dict__)失败， 
# 那么会调用到类的__getattr__函数, 如果没有定义这个函数，那么抛出AttributeError异常。
# 由此可见，__getattr__一定是作用于属性查找的最后一步，兜底。

# 当我们访问一个不存在的属性的时候，会抛出异常，提示我们不存在这个属性。
# 而这个异常就是__getattr__方法抛出的，其原因在于他是访问一个不存在的属性的最后落脚点，作为异常抛出的地方提示出错再适合不过了。

In [20]:
# 例1:

class A(object):
    def __init__(self, value):
        self.value = value
 
    def __getattr__(self, item):
        print(item)
        print("into __getattr__")
        return  "can not find"
 
a = A(10)
print(a.value)
print(a.name)

# 可以看出，访问存在的属性时，会正常返回值 
# 若该值不存在，则会进入最后的兜底函数__getattr__

10
name
into __getattr__
can not find


In [35]:
# 例2: 通过访问属性访问dict中的键值对

class ObjectDict(dict):
    def __init__(self, *args, **kwargs):
        super(ObjectDict, self).__init__(*args, **kwargs)  # super().__init__(*args, **kwargs) 表示继承了父类的init方法 

    def __getattr__(self, name):
        value =  self[name]
        if isinstance(value, dict):
            value = ObjectDict(value)
        return value

if __name__ == '__main__':
    od = ObjectDict(asf={'a': 1}, d=True, e=False)
    print(od.asf)
    print(od.asf.a)
    print(od.d)
    print(od.e)

{'a': 1}
1
True
False


In [16]:
# 例3: __getattr__使得实现adapter wrapper模式非常容易，
# 我们都知道“组合优于继承”，__getattr__实现的adapter就是以组合的形式。

class adaptee(object):
    def foo(self):
        print('foo in adaptee')
    def bar(self):
        print('bar in adaptee')

class adapter(object):
    def __init__(self):
        self.adaptee = adaptee()

    def foo(self):
        print('foo in adapter')
        self.adaptee.foo()

    def __getattr__(self, name):
        print('name',name)
        return getattr(self.adaptee, name)

if __name__ == '__main__':
    a = adapter()
    a.foo()
    a.bar()

foo in adapter
foo in adaptee
name bar
bar in adaptee


In [ ]:
# 2. __setattr__
# 在对一个属性设置值的时候，会调用到这个函数，每个设置值的方式都会进入这个方法。

In [44]:
# 在实例化的时候，会进行初始化，在__init__里，对value的属性值进行了设置，这时候会调用__setattr__方法。
# 在对a.value重新设置值100的时候，会再次进入__setattr__方法。

class A(object):
    def __init__(self, value):
        print("into __init__")
        self.value = value
 
    def __setattr__(self, name, value):
        print('name:', name)
        print('value:',value)
        print("into __setattr__")
        if value == 10:
            print("from __init__")
        object.__setattr__(self, name, value) 
 
a = A(10)
print(a.value)

into __init__
name: value
value: 10
into __setattr__
from __init__
10


In [42]:
a.value = 100
print(a.value)

name: value
value: 100
into __setattr__
100


In [36]:
# 需要注意的地方是，在重写__setattr__方法的时候千万不要重复调用造成死循环。
class A(object):
    def __init__(self, value):
        self.value = value
 
    def __setattr__(self, name, value):
        self.name = value

# 这是个死循环。当我们实例化这个类的时候，会进入__init__，
# 然后对value进行设置值，设置值会进入__setattr__方法，
# 而__setattr__方法里面又有一个self.name=value设置值的操作，会再次调用自身__setattr__，造成死循环。

In [40]:
# 除了上面调用object类的__setattr__避开死循环，还可以如下重写__setattr__避开循环。
class A(object):
    def __init__(self, value):
        self.value = value
 
    def __setattr__(self, name, value):
        self.__dict__[name] = value
 
a = A(10)
print(a.value)

10


In [46]:
# 编写一个Dict类，这个类的行为和dict一致，但是可以通过属性来访问
class Dict(dict):

    def __init__(self, **kw):
        super().__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

d = Dict(a=1, b=2)
print(d['a'])
print(d.b)

1
2


In [ ]:
# 3. __delattr__
# __delattr__是个删除属性的方法

In [50]:
class A(object):
    def __init__(self, value):
        self.value = value
 
    def __delattr__(self, item):
        object.__delattr__(self, item)
 
    def __getattr__(self, item):
        return "can not find attribute into __getattr__"

a = A(10)
print(a.value)

10


In [51]:
del(a.value)
print(a.value)

can not find attribute into __getattr__
